In [16]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier


In [2]:
data = pd.read_csv("DataSet_PCP_Change.csv")
data.head()

,testindex,outcome,distance,visit_count,claims_daysaway,tier,fqhc,pcp_lookback,family_assignment,kid,is_ped,same_gender,same_language,same_address
0,0,0,0.071912,0,NaN,2.0,0,0,0,0,0,1,0,0
1,2,0,0.448447,0,NaN,1.0,0,0,1,1,1,0,0,0
2,5,0,0.345793,0,NaN,1.0,0,1,0,0,0,1,0,0
3,6,0,1.857032,0,NaN,1.0,0,0,1,1,1,1,0,0
4,7,1,7.222060,0,NaN,1.0,0,0,1,1,0,0,0,0


In [3]:
data.shape

(3130, 14)

In [4]:
data.groupby(by='outcome', as_index=False).agg({'testindex': pd.Series.nunique})

,outcome,testindex
0,0,3003
1,1,127


In [5]:
data[data['outcome'] == 1].head()

,testindex,outcome,distance,visit_count,claims_daysaway,tier,fqhc,pcp_lookback,family_assignment,kid,is_ped,same_gender,same_language,same_address
4,7,1,7.222060,0,NaN,1.0,0,0,1,1,0,0,0,0
21,29,1,0.376928,5,0.0,1.0,0,0,0,1,0,0,0,0
25,37,1,6.557357,9,27.0,1.0,0,1,0,0,0,1,0,0
42,69,1,0.171770,0,NaN,2.0,0,0,0,0,0,0,0,0
72,116,1,0.303698,0,NaN,3.0,0,0,0,0,0,1,0,0


### Histogram to visualize all the possible features and the spread of their values.

In [6]:
# data.hist(figsize=(18,14))

In [7]:
# Remove rows with missing target, separate target from predictors
data.dropna(axis=0, subset=['outcome'], inplace=True)
y = data.outcome
data.drop(['outcome'], axis=1, inplace=True)

In [8]:
# replace all NaN with 0 from the remaining dataset
data = data.fillna(0)

In [9]:
data.head()

,testindex,distance,visit_count,claims_daysaway,tier,fqhc,pcp_lookback,family_assignment,kid,is_ped,same_gender,same_language,same_address
0,0,0.071912,0,0.0,2.0,0,0,0,0,0,1,0,0
1,2,0.448447,0,0.0,1.0,0,0,1,1,1,0,0,0
2,5,0.345793,0,0.0,1.0,0,1,0,0,0,1,0,0
3,6,1.857032,0,0.0,1.0,0,0,1,1,1,1,0,0
4,7,7.222060,0,0.0,1.0,0,0,1,1,0,0,0,0


In [10]:
y.head()

0    0
1    0
2    0
3    0
4    1
Name: outcome, dtype: int64

### Split the data in Training and Testing dataset using `train_test_split` method from sklearn.

In [11]:
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(data, y, train_size=0.8, test_size=0.2, random_state=0)

In [12]:
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, y_train)
preds = model.predict(X_valid)

In [13]:
mean_absolute_error(y_valid, preds)

0.07432907348242812

In [14]:
LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
preds = model.predict(X_valid)

mean_absolute_error(y_valid, preds)

/opt/miniconda3/envs/bits-py-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.038338658146964855

In [17]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_valid)

In [19]:
mean_absolute_error(y_valid, preds)

0.08306709265175719